In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
USE IDENTIFIER("fhir_workshop." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
SHOW VOLUMES;

In [0]:
DECLARE OR REPLACE VARIABLE list_stmnt STRING;

SET VARIABLE list_stmnt = "LIST '/Volumes/fhir_workshop/" || schema_use || "/landing/';";

SELECT list_stmnt;

In [0]:
EXECUTE IMMEDIATE list_stmnt;

In [0]:
DROP TABLE IF EXISTS fhir_bronze;

In [0]:
DECLARE OR REPLACE VARIABLE create_streaming_bronze_stmnt STRING;

SET VARIABLE create_streaming_bronze_stmnt = "
CREATE OR REFRESH STREAMING TABLE fhir_bronze (
  file_metadata STRUCT<
    file_path: STRING,
    file_name: STRING,
    file_size: BIGINT,
    file_block_start: BIGINT,
    file_block_length: BIGINT,
    file_modification_time: TIMESTAMP
  > COMMENT 'Original meta date of the file ingested from the volume.'
  ,value STRING COMMENT 'Original JSON record ingested from the volume as a full text string value.'
)
COMMENT 'Ingest FHIR JSON records as Full Text STRING'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true',
  'delta.enableDeletionVectors' = 'true',
  'delta.enableRowTracking' = 'true',
  'quality' = 'bronze'
)
AS SELECT
  _metadata as file_metadata
  ,* 
FROM STREAM read_files(
  '/Volumes/fhir_workshop/" || schema_use || "/landing/'
  ,format => 'text'
  ,wholeText => true
)
";

SELECT create_streaming_bronze_stmnt;

In [0]:
EXECUTE IMMEDIATE create_streaming_bronze_stmnt;

In [0]:
SELECT * FROM fhir_bronze limit 1;

In [0]:
select count(*) as rcrd_cnt from fhir_bronze;

In [0]:
DROP TABLE IF EXISTS fhir_bronze_variant;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_bronze_variant (
  file_metadata STRUCT<
    file_path: STRING,
    file_name: STRING,
    file_size: BIGINT,
    file_block_start: BIGINT,
    file_block_length: BIGINT,
    file_modification_time: TIMESTAMP
  > COMMENT 'Original meta date of the file ingested from the volume.'
  ,bundle_uuid STRING COMMENT 'Unique identifier for the FHIR bundle.'
  ,fhir VARIANT COMMENT 'Original JSON record fully parsed as a variant data type.'
)
COMMENT 'Evaluate FHIR JSON records as VARIANT'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true' 
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  file_metadata
  ,uuid() as bundle_uuid
  ,try_parse_json(value) as fhir 
FROM STREAM fhir_bronze;